In [4]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score, classification_report, zero_one_loss
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.linear_model import SGDClassifier  # For decision boundary visualization
from sklearn.linear_model import LogisticRegression  # For main logistic regression models
from sklearn.multioutput import MultiOutputClassifier

import pandas as pd
import warnings
warnings.filterwarnings('ignore')